In [5]:
import pandas as pd
import pickle
import xgboost as xgb
import numpy as np

from matplotlib import pyplot as plt
import tensorflow as tf

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
tf.config.list_physical_devices("GPU")

[]

In [6]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [7]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16271939443976034094
 xla_global_id: -1]

In [24]:
import pandas as pd
# df =pd.read_csv("0_jy_interpol.csv")
# nwp_cols = "lead_hr,NDNSW,SWDIR,SWDIF,TDSWS,NDNLW,OULWT,DLWS,NCPCP,SNOL,LSPRATE,LSSRATE,HPBLA,HFSOIL,LLRIB,UGRD,VGRD,HFSFC,XBLWS,YBLWS,LHTFL,TMP,TMIN,TMAX,SPFH,TOMFS,RH,VIS,FOGFR,DPT,PVIS5,15TL,15QT,VISIP,SHFLT-1,SHFLT-2,SHFLT-3,SHFLT-4,SHFLT-5,SHFLT-6,SHFLT-7,SHFLT-8,SHFLT-9,HTBM,SCST,DSCNC,DSCOC,NETTL-1,NETTL-2,NETTL-3,NETTL-4,NETTL-5,NETTL-6,NETTL-7,NETTL-8,NETTL-9,STOT-1,STOT-2,STOT-3,STOT-4,STOT-5,STOT-6,STOT-7,STOT-8,STOT-9,HCNP-1,HCNP-2,HCNP-3,HCNP-4,HCNP-5,WCNP-1,WCNP-2,WCNP-3,WCNP-4,WCNP-5,T15T-1,T15T-2,T15T-3,T15T-4,T15T-5,T15T-6,T15T-7,T15T-8,T15T-9,SH15-1,SH15-2,SH15-3,SH15-4,SH15-5,SH15-6,SH15-7,SH15-8,SH15-9,HDLB,STCP-1,STCP-2,STCP-3,STCP-4,STCP-5,MAXGUST,CBLT,SNOM,SMCL-1,SMCL-2,SMCL-3,SMCL-4,TSOIL-1,TSOIL-2,TSOIL-3,TSOIL-4,ROFR,SROFR,VLCDC,LCDC,MCDC,HCDC,CB25,CB45,TCAR,TCAM,CFB10,LCB,WBFLH,TCTH,50MU-min,50MV-min,50MU-max,50MV-max,PRMSL,SNOAL,TMP-SFC,HPBL,SFCR,LAND,FRICE,DIST,PRES"
df = pd.read_csv("clean_dataset/s1_case1.csv")
nwp_cols="lead_hr,NDNSW,SWDIR,SWDIF,TDSWS,NDNLW,OULWT,DLWS,HPBLA,HFSOIL,LLRIB,UGRD,VGRD,HFSFC,XBLWS,YBLWS,LHTFL,TMP,TMIN,TMAX,SPFH,TOMFS,RH,VIS,DPT,15TL,15QT,VISIP,SHFLT-1,SHFLT-2,SHFLT-3,SHFLT-4,SHFLT-5,SHFLT-6,SHFLT-7,SHFLT-8,HTBM,NETTL-1,NETTL-2,NETTL-3,NETTL-4,NETTL-5,NETTL-6,NETTL-7,NETTL-8,STOT-1,STOT-2,STOT-3,STOT-4,STOT-5,STOT-6,STOT-7,STOT-8,STOT-9,T15T-1,T15T-2,T15T-3,T15T-4,T15T-8,SH15-1,SH15-2,SH15-3,SH15-4,SH15-8,STCP-1,STCP-2,STCP-3,STCP-4,MAXGUST,CBLT,SNOM,SMCL-1,SMCL-2,SMCL-3,SMCL-4,TSOIL-1,TSOIL-2,TSOIL-3,TSOIL-4,ROFR,SROFR,VLCDC,LCDC,MCDC,HCDC,TCAR,TCAM,LCB,TCTH,50MU-min,50MV-min,50MU-max,50MV-max,PRMSL,SNOAL,TMP-SFC,HPBL,PRES"
nwp_cols = nwp_cols.split(',')
output_col = ["OT"]
FCST_cols =['date']
# CRTN_cols = ['CRTN_TM']
all_cols = FCST_cols+nwp_cols+output_col
tmp_df= df[all_cols]

tmp_df.date = pd.to_datetime(tmp_df.date)

#case1 
# case11_df= tmp_df.copy()
# case11_df=case11_df[case11_df['lead_hr']!=0]
# # case11_df=case11_df.drop(columns=['CRTN_TM'])
# case11_df.reset_index(inplace=True)
# case11_df=case11_df.drop(columns=['index'])


# case2
# tmp_df['day']=tmp_df.date.dt.day
# case11_df= tmp_df.copy()
# case11_df=case11_df[case11_df['lead_hr']!=0]
# train_df = case11_df[case11_df['day']<=25]
# test_df = case11_df[case11_df['day']>25]
# case11_df = pd.concat([train_df,test_df])
# case11_df=case11_df.drop(columns=['day'])
# case11_df.reset_index(inplace=True)
# case11_df=case11_df.drop(columns=['index'])

# case3 
case11_df= tmp_df.copy()
case11_df=case11_df[case11_df['lead_hr']!=0]
case11_df['month'] = case11_df.date.dt.month
case11_df['day']=case11_df.date.dt.day
test_df=case11_df[(case11_df['month'].isin([5,8,11,2]))&(case11_df['day'] > 12)]
train_df=case11_df.drop(index=list(test_df.index))
case11_df = pd.concat([train_df,test_df])
case11_df=case11_df.drop(columns=['day','month'])
case11_df.reset_index(inplace=True)
case11_df=case11_df.drop(columns=['index'])


In [25]:
case11_df=case11_df.set_index("date")

In [26]:

from sklearn.preprocessing import StandardScaler

# sc = StandardScaler() 
# x = case11_df.drop(['OT'], axis=1).values # 독립변인들의 value값만 추출
# y = case11_df['OT'].values # 종속변인 추출
# x_std = sc.fit_transform(x) # standardizing the data
# x_std_df= pd.DataFrame(x_std, columns=case11_df.columns)

# case11_df=case11_df.reset_index()
# case11_df=case11_df[['date','OT']]
# case11_df=pd.merge(case11_df,x_std_df,how='inner',left_index=True,right_index=True)
# case11_df.head()

sc = StandardScaler()
x = case11_df.values
x_std = sc.fit_transform(x)
x_std_df = pd.DataFrame(x_std, columns = case11_df.columns)

case11_df = case11_df.reset_index()
case11_df=case11_df.reset_index()
case11_df=case11_df[['date']]
case11_df=pd.merge(case11_df,x_std_df,how='inner',left_index=True,right_index=True)


In [27]:
def feat_importance(input_cols, model):
    results=pd.DataFrame()
    results['columns']= input_cols
    results['importances'] = model.feature_importances_
    results.sort_values(by='importances', ascending=False, inplace=True)

    print(results[:20])

In [28]:
all_len = len(case11_df)

In [29]:
# data.CRTN_TM = pd.to_datetime(data.CRTN_TM)
case11_df.date = pd.to_datetime(case11_df.date)
# data = data[data.OT > 0]

training_original_df = case11_df[:int(all_len*0.8)].reset_index(drop=True)
test_original_df = case11_df[int(all_len*0.8):].reset_index(drop=True)
test_original_df=test_original_df[(test_original_df['date'].dt.hour>=7) &(test_original_df['date'].dt.hour<20)]
test_original_df=test_original_df.reset_index(drop=True)

training_df = training_original_df[FCST_cols+nwp_cols+output_col]

train_X = training_df[nwp_cols].values
train_y = training_df['OT'].values

test_X = test_original_df[nwp_cols].values
test_df = test_original_df




In [30]:
import xgboost as xgb
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
start_dt = datetime.now()
model = xgb.XGBRegressor()

model.fit(train_X, train_y)

feat_importance(nwp_cols, model)

    columns  importances
37  NETTL-1     0.507534
40  NETTL-4     0.135360
96     HPBL     0.086611
4     TDSWS     0.028055
21    TOMFS     0.022659
30  SHFLT-3     0.016766
36     HTBM     0.014885
75  TSOIL-1     0.013458
44  NETTL-8     0.008826
5     NDNLW     0.008762
39  NETTL-3     0.007544
16    LHTFL     0.006367
86     TCAM     0.005321
50   STOT-6     0.005271
28  SHFLT-1     0.005054
64   STCP-1     0.005050
3     SWDIF     0.004855
18     TMIN     0.004425
71   SMCL-1     0.003683
19     TMAX     0.003450


In [31]:
test_start_dt = datetime.now()
predictions = model.predict(test_X)

prediction_df = pd.DataFrame(predictions, columns=['OT'])

# prediction_df[prediction_df['OT']<=0]=0
test_df["OT_pred"] = prediction_df

# print(test_df)
from sklearn.metrics import mean_absolute_error,mean_squared_error
mse = mean_squared_error(test_df['OT'], test_df['OT_pred'])
print(mse)
mae=mean_absolute_error(test_df['OT'], test_df['OT_pred'])
print(mae)

print((test_start_dt- start_dt).total_seconds())
print((datetime.now()-test_start_dt).total_seconds())

0.1942789410246862
0.2920341126416739
6.978647
0.009516
